- Daten einlesen
- Texte bereinigen
- Modell(e) trainieren
- Accuracy, Precision & Recall berechnen


In [3]:
%pip install pandas


Note: you may need to restart the kernel to use updated packages.


### Schritt 1A: Einlesen der Rohdaten
In diesem Schritt wird geprüft, ob alle Urteilstexte aus dem bereitgestellten OpenJur-Datensatz korrekt geladen werden können. Ziel ist es, die Datengrundlage zu verifizieren, bevor weitere Verarbeitungsschritte erfolgen.

In [4]:
import os
import pandas as pd

DATA_DIR = "../data/Gerichtsurteile_Openjur" 
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".txt")]

print("Pfad:", os.path.abspath(DATA_DIR))
print("Anzahl .txt:", len(files))
print("Erste 10 Dateien:", files[:10])


Pfad: c:\Users\humme\OneDrive\Dokumente\Uni Ulm\ds_law\backend\data\Gerichtsurteile_Openjur
Anzahl .txt: 2375
Erste 10 Dateien: ['2090187.txt', '2112111.txt', '2112115.txt', '2112117.txt', '2112118.txt', '2112119.txt', '2112121.txt', '2112123.txt', '2124977.txt', '2126821.txt']


Die Ausgabe bestätigt, dass insgesamt 2375 Urteilstexte erfolgreich eingelesen wurden. Die Dateinamen entsprechen den von OpenJur vergebenen Fall-IDs, wodurch die Konsistenz und Vollständigkeit der Datengrundlage sichergestellt ist.

### Schritt 1B: Strukturierung der Urteilstexte
Im nächsten Schritt werden die eingelesenen Urteilstexte in eine tabellarische Datenstruktur überführt. Ziel ist es, jede gerichtliche Entscheidung als einzelne Beobachtung abzubilden und damit eine Grundlage für die spätere Merkmalsextraktion und Modellierung zu schaffen.

In [5]:
def read_txt(path: str) -> str:
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    text = read_txt(os.path.join(DATA_DIR, fn))
    rows.append({
        "case_id": case_id,
        "text": text
    })

df = pd.DataFrame(rows)
df["text_len"] = df["text"].str.len()

print("DataFrame shape:", df.shape)
df.head(3)


DataFrame shape: (2375, 3)


,case_id,text,text_len
0,2090187,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,8774
1,2112111,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,47492
2,2112115,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...,35964


Die resultierende Datenstruktur umfasst 2375 gerichtliche Entscheidungen. Jede Zeile entspricht einem Urteil, das eindeutig über eine Fall-ID referenziert ist. Zusätzlich wurde die Textlänge der Urteile berechnet, um eine Plausibilitätsprüfung der Datengrundlage zu ermöglichen.

### Schritt 1C: Einschränkung auf Entscheidungen der Landgerichte

Gemäß der Aufgabenstellung wird der Datensatz auf Urteile deutscher Landgerichte beschränkt. Hintergrund ist, dass unterschiedliche
Gerichtsebenen teils abweichende rechtliche Maßstäbe anwenden, was zu inkonsistenten Lernsituationen für das Modell führen kann. 
Zunächst wird geprüft, ob es sich bei der Entscheidung um ein Urteil eines deutschen Landgerichts handelt. Anschließend wird der thematische Bezug zum Dieselskandal anhand zentraler Schlüsselbegriffe identifiziert. Durch diese getrennte Markierung wird Transparenz über die Zusammensetzung des Datensatzes geschaffen und eine spätere Anpassung der Filterlogik ermöglicht, ohne frühzeitig Daten zu verwerfen.

In [6]:
lg_pattern = r"\bLandgericht\b|\bLG\s+[A-ZÄÖÜa-zäöü]+"

df["is_landgericht"] = df["text"].str.contains(
    lg_pattern, case=False, regex=True
)

df["is_landgericht"].value_counts()


is_landgericht
True     2316
False      59
Name: count, dtype: int64

In [7]:
keywords = [
    "diesel",
    "abschalt",
    "abgasskandal",
    "unzulässige abschalteinrichtung",
    "schadensersatz"
]

pattern = "|".join(keywords)

df["is_diesel_case"] = df["text"].str.lower().str.contains(
    pattern, regex=True
)

df["is_diesel_case"].value_counts()


is_diesel_case
True     2371
False       4
Name: count, dtype: int64

### Schritt 1D: Finaler Analyse-Datensatz 

In [8]:
df_analysis = df[df["is_diesel_case"] & df["is_landgericht"]].copy()
df_analysis.shape


(2313, 5)

Zur methodischen Absicherung wurde überprüft, ob alle Dokumente entsprechende Schlüsselbegriffe enthalten. Die Analyse bestätigt, dass nahezu alle Urteile einen expliziten Bezug zum Dieselskandal aufweisen.

### Schritt 2: Definition der Zielvariable

Ziel der Analyse ist die Vorhersage, ob in einem Urteil ein Schadensersatz zugesprochen wurde oder nicht. 
Die Zielvariable wird binär modelliert (1 = Schadensersatz zugesprochen, 0 = kein Schadensersatz). 
Da strukturierte Labels nicht vorliegen, erfolgt die Ableitung regelbasiert anhand typischer Formulierungen im Entscheidungstext. Die Heuristik wird anschließend stichprobenartig überprüft.


In [9]:
import re

def infer_target(text: str):
    t = text.lower()

 # Verurteilung zur Zahlung → Schadensersatz
    positive_patterns = [
        r"wird verurteilt.*zu zahlen",
        r"wird verurteilt.*schadensersatz",
        r"hat .* schadensersatz .* zu zahlen"
    ]
    if any(re.search(p, t, flags=re.DOTALL) for p in positive_patterns):
        return 1
    
    # Klage abgewiesen → kein Schadensersatz
    negative_patterns = [
        "die klage wird abgewiesen",
        "klage wird abgewiesen",
        "wird abgewiesen"
    ]
    if any(p in t for p in negative_patterns):
        return 0

    # unklar
    return None


df_analysis["target"] = df_analysis["text"].apply(infer_target)
df_analysis["target"].value_counts(dropna=False)


target
1.0    1486
NaN     497
0.0     330
Name: count, dtype: int64

In [10]:
df_analysis[df_analysis["target"].notna()][["case_id","target","text"]].sample(20, random_state=42)


,case_id,target,text
1971,2447936,0.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
873,2297505,1.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
184,2175373,1.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
525,2241959,1.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
769,2294874,1.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
2259,2470452,1.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
385,2203672,1.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
2001,2448961,0.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
1769,2391516,1.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...
1140,2319466,0.0,Rechtsprechung\n\t\t\t\t\t\tAktuell\n\t\t\t\t\...


**Umgang mit unklaren Fällen**

Nicht alle Urteile enthalten eine eindeutig identifizierbare Tenorformulierung, aus der zweifelsfrei hervorgeht, ob ein Schadensersatz zugesprochen wurde oder nicht (z. B. bei Vergleichen oder rein prozessualen Entscheidungen).  
Diese Fälle werden in der Zielvariable als *unklar* (`None`) gekennzeichnet und für die weitere Modellierung ausgeschlossen.  
Durch diese Einschränkung wird sichergestellt, dass das Modell ausschließlich auf eindeutig gelabelten Entscheidungen trainiert wird und Verzerrungen durch unsichere Zuordnungen vermieden werden.